## New main file

In [ ]:
import pandas as pd
from MJP.majority_portfolio_utilities_TV import * 
from MJP.myplotMJ import * 

date_initial='2000-01-01'
date_final='2023-12-31'

# if True delete stocks lower than 5th and higher than 95th NYSE ME percentile
remove_outliers = False # False True
inf=None
sup=None

factors_df=pd.DataFrame({
    'factors': ['ag','beta','bm','cumret','dolvol6','gp','ill6','ns','size','volatility','acc'],
    'signs': [  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.],
    'wsigns':[  -1.,  1.,   1.,    1.,     -1.,     1.,    1.,  -1.,  -1.,   -1.,   -1.]
})



df=pd.read_csv('monthly_data.csv')
df['date']=pd.to_datetime(df['date'])


## configurazione base:
MJ_configuration={
    'K' : 12, # holding_periods in months
    'lag' : 5, # 5 for june reallocation
    'factors' : factors_df['factors'].to_list(),
    'num_port' : 10, # number of portfolios
    'num_cat' : 6, # number of categorical variables to use per factor in mj #'num_cat': 6
    'weighting': True,
    'verbose': True,
    'n_jobs':-1,
    'mj_window':1,
    'method' : 'majority', # mean_rank, majority, 75q, 90q, lex, dlex
    'rolling_method':'profile',  ###'rank','vote','profile'
    'treat_na_mj':'median',
    'remove_outliers':remove_outliers,
    'inclusive':True,
    'fix_signs':True, # True: tieni fisso il numero di votanti ed il loro segno
    'all_voters_not_nan_on_reallocation':True,
    #### PARAMETRI UTILIZZATI SOLO SE fix_signs=False ####
    'min_voters':5,    
    'voting_window':6,
    'sign_voting_window':12,
    'p_threshold' : 0.1, 
    'delta_utility' : 0,
    'eliminations': 1,
    'players_batch_size': 5,
    'small': True
}

if remove_outliers:
    MJ_configuration['outliers']=[inf,sup]

if MJ_configuration['weighting']:
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['signs'].to_list()
    compute='mj'
elif not MJ_configuration['weighting']:
    MJ_configuration['default_voters']=factors_df['factors'].to_list()
    MJ_configuration['default_signs']=factors_df['wsigns'].to_list()
    compute='wmj'


factors=factors_df['factors'].to_list()


In [ ]:
import matplotlib.pyplot as plt

### COMPUTE SINGLE FACTOR STRATEGIES
portfolios, weighted_portfolios, portfolios_stock_reallocation = compute_factor_strategies(df, MJ_configuration)


### COMPUTE EQUAL WEIGHTED MJ STRATEGY
MJ_configuration['weighting']=False
MJ_portfolios, \
mj_voters, \
MJ_portfolios_stock_reallocation = compute_MJ_portfolio_strategy(df, 
                                                            MJ_configuration)


### COMPUTE VALUE WEIGHTED MJ STRATEGY
MJ_configuration['weighting']=True
MJ_weighted_portfolios, \
mj_voters, \
MJ_weighted_portfolios_stock_reallocation = compute_MJ_portfolio_strategy(df, 
                                                            MJ_configuration)

### MERGE PORTFOLIO RESULTS
portfolios['mj']=MJ_portfolios['mj']
weighted_portfolios['wmj']=MJ_weighted_portfolios['wmj']

### MERGE REALLOCATION RESULTS
portfolios_stock_reallocation['mj']={}
portfolios_stock_reallocation['wmj']={}
portfolios_stock_reallocation['mj']['EW_turnover']=MJ_portfolios_stock_reallocation['mj']
portfolios_stock_reallocation['wmj']['VW_turnover']=MJ_weighted_portfolios_stock_reallocation['wmj']


In [ ]:
#PLOT THE RESULTS OF SINGLA FACTOR SORTING FOR A GIVEN FACTOR

factor='ns'
((1+portfolios[factor]).cumprod()-1).plot()

In [ ]:
# Plot cumulative returns of EW MJ portfolios 
for port in [f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']:
    plt.plot(((1+portfolios['mj'])[port].cumprod()-1),label=port)
plt.legend()
plt.show()

# Plot cumulative returns of VW MJ portfolios 
for port in [f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']:
    plt.plot(((1+weighted_portfolios['wmj'])[port].cumprod()-1),label=port)
plt.legend()
plt.show()


In [ ]:
list_reallocation={}
list_reallocation['even']=[]
list_reallocation['weighted']=[]


for factor in factors+['mj']:
    for port in range(1,MJ_configuration['num_port']+1):
        list_reallocation['even'].append([factor,port,portfolios_stock_reallocation[factor]['EW_turnover'].iloc[int(port)-1].iloc[1]])
        
for factor in factors+['wmj']:
    for port in range(1,MJ_configuration['num_port']+1):
        list_reallocation['weighted'].append([factor,port,portfolios_stock_reallocation[factor]['VW_turnover'].iloc[int(port)-1].iloc[1]])
        

list_reallocation['even']=pd.DataFrame(list_reallocation['even'])
list_reallocation['even'].columns=['factor','portfolio','reallocation rate']

list_reallocation['weighted']=pd.DataFrame(list_reallocation['weighted'])
list_reallocation['weighted'].columns=['factor','portfolio','reallocation rate']

reshaped_EW_reallocation = list_reallocation['even'].pivot(index='factor', columns='portfolio', values='reallocation rate')
reshaped_EW_reallocation.columns=[f'port{int(i)}' for i in reshaped_EW_reallocation.columns]
reshaped_VW_reallocation = list_reallocation['weighted'].pivot(index='factor', columns='portfolio', values='reallocation rate')
reshaped_VW_reallocation.columns=[f'port{int(i)}' for i in reshaped_VW_reallocation.columns]


In [ ]:

from MJP.myplotMJ import * 
mybarplot(list_reallocation['even'], portfolio='portfolio', data_col='reallocation rate', factor='factor', nameplotx ='EW portfolios',
          save='Y', file_format='pdf', save_as=f'EW_reallocation_rates.pdf')

reshaped_EW_reallocation
print(reshaped_EW_reallocation.to_latex(float_format=lambda x: '{:.2f}'.format(x)))

In [ ]:
mybarplot(list_reallocation['weighted'], portfolio='portfolio', data_col='reallocation rate', factor='factor', nameplotx ='VW portfolios',
          save='Y', file_format='pdf', save_as=f'VW_reallocation_rates.pdf')

print(reshaped_VW_reallocation.to_latex(float_format=lambda x: '{:.2f}'.format(x)))

# Main statistics and latex table

In [ ]:
# ========================================================
# T-Value and P-Value
# ========================================================
# l’ipotesi nulla afferma che qualsiasi differenza tra la media della popolazione e la media campionaria 
# è dovuta al caso. Se il valore p del test t è inferiore a una certa soglia (ad esempio 0,05), 
# allora si può rifiutare l’ipotesi nulla e concludere che esiste una differenza significativa 
# tra la media della popolazione e la media campionaria. Altrimenti, non si può rifiutare l’ipotesi nulla 
# e si conclude che non c’è alcuna differenza significativa tra la media della popolazione e la media campionaria 
from scipy import stats

sharpe_ratios={}
sharpe_ratios['even']=[]
sharpe_ratios['weighted']=[]

volatilities={}
volatilities['even']=[]
volatilities['weighted']=[]

returns={}
returns['even']=[]
returns['weighted']=[]




for factor in factors+['mj']:
    sharpe_ratio_factor=np.sqrt(12)*(portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']]).mean()\
                                    /portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']].std()
    volatility_factor=np.sqrt(12)*(portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']].std())
    return_factor=12*(portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']].mean())
    for i in range(len(sharpe_ratio_factor.index)):
        if i!=len(sharpe_ratio_factor.index)-1: 
            sharpe_ratios['even'].append([factor,i+1,sharpe_ratio_factor.iloc[i]])
            volatilities['even'].append([factor,i+1,volatility_factor.iloc[i]])
            returns['even'].append([factor,i+1,return_factor.iloc[i]])
        else:
            sharpe_ratios['even'].append([factor,'long_short',sharpe_ratio_factor.iloc[i]])
            volatilities['even'].append([factor,'long_short',volatility_factor.iloc[i]])
            returns['even'].append([factor,'long_short',return_factor.iloc[i]])

sharpe_ratios['even']=pd.DataFrame(sharpe_ratios['even'])
sharpe_ratios['even'].columns=['factor','portfolio','sharpe ratio']
                                   
volatilities['even']=pd.DataFrame(volatilities['even'])
volatilities['even'].columns=['factor','portfolio','volatility']

returns['even']=pd.DataFrame(returns['even'])
returns['even'].columns=['factor','portfolio','returns']

for factor in factors+['wmj']:
    sharpe_ratio_factor=np.sqrt(12)*(weighted_portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']]).mean()\
                                    /weighted_portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']].std()
    volatility_factor=np.sqrt(12)*(weighted_portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']].std())
    return_factor=12*(weighted_portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']].mean())
    for i in range(len(sharpe_ratio_factor.index)):
        if i!=len(sharpe_ratio_factor.index)-1: 
            sharpe_ratios['weighted'].append([factor,i+1,sharpe_ratio_factor.iloc[i]])
            volatilities['weighted'].append([factor,i+1,volatility_factor.iloc[i]])
            returns['weighted'].append([factor,i+1,return_factor.iloc[i]])
        else:
            sharpe_ratios['weighted'].append([factor,'long_short',sharpe_ratio_factor.iloc[i]])
            volatilities['weighted'].append([factor,'long_short',volatility_factor.iloc[i]])
            returns['weighted'].append([factor,'long_short',return_factor.iloc[i]])

sharpe_ratios['weighted']=pd.DataFrame(sharpe_ratios['weighted'])
sharpe_ratios['weighted'].columns=['factor','portfolio','sharpe ratio']
                                   
                                                                      
volatilities['weighted']=pd.DataFrame(volatilities['weighted'])
volatilities['weighted'].columns=['factor','portfolio','volatility']

returns['weighted']=pd.DataFrame(returns['weighted'])
returns['weighted'].columns=['factor','portfolio','returns']

t_stat={}
p_val ={}
t_stat['even']=[]
p_val['even']=[]

t_stat['weighted']=[]
p_val['weighted']=[]


for factor in factors+['mj']:
    t = (stats.ttest_1samp(portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']],0.0))

    for i in range(len(t.statistic)):
        if i!=len(t.statistic)-1: 
            t_stat['even'].append([factor,i+1,t.statistic[i]])
            p_val['even'].append([factor,i+1,t.pvalue[i]])
        else:
            t_stat['even'].append([factor,'long_short',t.statistic[i]])
            p_val['even'].append([factor,'long_short',t.pvalue[i]])
            
t_stat['even']=pd.DataFrame(t_stat['even'])
t_stat['even'].columns=['factor','portfolio','t-stat']
p_val['even']=pd.DataFrame( p_val['even'])
p_val['even'].columns=['factor','portfolio','p-value']
#ewp_ttest = pd.merge(t_stat['even'],p_val['even'],  on=['factor','portfolio'], how='left')

for factor in factors+['wmj']:
    t = (stats.ttest_1samp(weighted_portfolios[factor][[f'port{i}' for i in range(1,MJ_configuration['num_port']+1)]+['long_short']],0.0))

    for i in range(len(t.statistic)):
        if i!=len(t.statistic)-1: 
            t_stat['weighted'].append([factor,i+1,t.statistic[i]])
            p_val['weighted'].append([factor,i+1,t.pvalue[i]])
        else:
            t_stat['weighted'].append([factor,'long_short',t.statistic[i]])
            p_val['weighted'].append([factor,'long_short',t.pvalue[i]])
            
t_stat['weighted']=pd.DataFrame(t_stat['weighted'])
t_stat['weighted'].columns=['factor','portfolio','t-stat']
p_val['weighted']=pd.DataFrame( p_val['weighted'])
p_val['weighted'].columns=['factor','portfolio','p-value']

In [ ]:
ewp_stat = pd.merge(returns['even'],t_stat['even'],  on=['factor','portfolio'], how='left')
ewp_stat = pd.merge(ewp_stat,volatilities['even'],  on=['factor','portfolio'], how='left')
ewp_stat = pd.merge(ewp_stat,sharpe_ratios['even'],  on=['factor','portfolio'], how='left')

vwp_stat = pd.merge(returns['weighted'], t_stat['weighted'],  on=['factor','portfolio'], how='left')
vwp_stat = pd.merge(vwp_stat,volatilities['weighted'],  on=['factor','portfolio'], how='left')
vwp_stat = pd.merge(vwp_stat,sharpe_ratios['weighted'],  on=['factor','portfolio'], how='left')

In [ ]:
# Trasforma il DataFrame
df_melt = ewp_stat.melt(id_vars=['factor', 'portfolio'], var_name='variable', value_name='value')

# Crea una nuova tabella pivot
ew_tab = df_melt.pivot_table(index=['factor', 'variable'], columns='portfolio', values='value', sort=False)
ew_tab = ew_tab.rename(columns={'long_short': 'High-Low'})
#ew_tab

# Trasforma il DataFrame
df_melt = vwp_stat.melt(id_vars=['factor', 'portfolio'], var_name='variable', value_name='value')

# Crea una nuova tabella pivot
vw_tab = df_melt.pivot_table(index=['factor', 'variable'], columns='portfolio', values='value', sort=False)
vw_tab = vw_tab.rename(columns={'long_short': 'High-Low'})
#vw_tab

In [ ]:
vw_tab

In [ ]:
def highlight_rows(s, names):
    is_in_list = s.name[0] in names
    return ['background-color: aliceblue' if is_in_list else '' for v in s]

names1 = ['beta','bm','cumret','gp','ill6','wmj']  # replace with your list of names
names2 = ['ag','dolvol6','ns','size', 'volatility','acc'] 

ew_tab1 = ew_tab.style.apply(highlight_rows, subset=[10], args=(names1,), axis=1)
ew_tab1 = ew_tab1.apply(highlight_rows, subset=[1], args=(names2,), axis=1)

vw_tab1 = vw_tab.style.apply(highlight_rows, subset=[10], args=(names1,), axis=1)
vw_tab1 = vw_tab1.apply(highlight_rows, subset=[1], args=(names2,), axis=1)

ew_p_val =p_val['even'].pivot_table(index=['factor'], columns='portfolio', values='p-value')
ew_p_val = ew_p_val.rename(columns={'long_short': 'High-Low'})

vw_p_val =p_val['weighted'].pivot_table(index=['factor'], columns='portfolio', values='p-value')
vw_p_val = vw_p_val.rename(columns={'long_short': 'High-Low'})



def format_t_statistic(df, pvalue_df):
    # Creiamo una copia del dataframe per non modificarlo
    df_copy = df.copy()
    for index, row in df.iterrows():
        # Otteniamo il fattore corrispondente (ag o beta)
        factor = index[0]

        # Iteriamo su ogni elemento della riga
        for col in row.index:
            # Otteniamo il valore e il corrispondente p-value
            value = row[col]
            p_value = pvalue_df.loc[factor, col]
            

            # Se l'indice contiene 'returns'
            if 'returns' in index:
                df_copy.loc[index, col] = '{:.4f}'.format(value)
                # Se il p-value è minore di 0.05, mettiamo il t-stat in grassetto e tra parentesi
                if p_value < 0.05:
                    df_copy.loc[index, col] = '\\textbf{{{:.4f}}}'.format(value)
            elif 'sharpe ratio' in index or 'volatility' in index:
                df_copy.loc[index, col] = '{:.4f}'.format(value)
                
            if 't-stat' in index:
                df_copy.loc[index, col] = '({:.3f})'.format(value)
    return df_copy
tabew = format_t_statistic(ew_tab, ew_p_val)
tabvw = format_t_statistic(vw_tab, vw_p_val)



EWtab1=tabew.iloc[:, :1]
EWtab10=tabew.iloc[:, 9:]
EWtt = EWtab1.join(EWtab10).reset_index()
EWtt.index.name = None

EWtt = EWtt.melt(id_vars=['factor', 'variable'], 
                    value_vars=[1, 10, 'High-Low'], 
                    var_name='port', value_name='value')

condition_1 = EWtt['factor'].isin(['ag', 'dolvol6','ns','size','volatility','acc']) & (EWtt['port'] == 10)
condition_10 = EWtt['factor'].isin(['beta', 'bm','cumret','gp','ill6','mj']) & (EWtt['port'] == 1)
EWtt = EWtt[~(condition_1 | condition_10)]
EWtt['port'] = EWtt['port'].replace({1: 'best', 10: 'best'})


EWpivot_table = EWtt.pivot_table(index=['port', 'variable'], 
                             columns='factor', 
                             values='value', 
                             aggfunc='first',  # or 'sum' for string concatenation
                             sort=False)
EWpivot_table = EWpivot_table.reindex(sorted(EWpivot_table.columns), axis=1)
new_cols = [col for col in EWpivot_table.columns if col != 'mj'] + ['mj']
EWpivot_table = EWpivot_table[new_cols]

VWtab1=tabvw.iloc[:, :1]
VWtab10=tabvw.iloc[:, 9:]
VWtt = VWtab1.join(VWtab10).reset_index()
VWtt.index.name = None

VWtt = VWtt.melt(id_vars=['factor', 'variable'], 
                    value_vars=[1, 10, 'High-Low'], 
                    var_name='port', value_name='value')

condition_1 = VWtt['factor'].isin(['ag', 'dolvol6','ns','size','volatility','acc']) & (VWtt['port'] == 10)
condition_10 = VWtt['factor'].isin(['beta', 'bm','cumret','gp','ill6','wmj']) & (VWtt['port'] == 1)
VWtt = VWtt[~(condition_1 | condition_10)]
VWtt['port'] = VWtt['port'].replace({1: 'best', 10: 'best'})


VWpivot_table = VWtt.pivot_table(index=['port', 'variable'], 
                             columns='factor', 
                             values='value', 
                             aggfunc='first',  # or 'sum' for string concatenation
                             sort=False)

VWpivot_table = VWpivot_table.reindex(sorted(VWpivot_table.columns), axis=1)
print(EWpivot_table.to_latex())
print(VWpivot_table.to_latex())

EWpivot_table

In [ ]:

ew_tab1 = tabew.style.apply(highlight_rows, subset=[10], args=(names1+['mj'],), axis=1)
ew_tab1 = ew_tab1.apply(highlight_rows, subset=[1], args=(names2,), axis=1)

vw_tab1 = tabvw.style.apply(highlight_rows, subset=[10], args=(names1+['wmj'],), axis=1)
vw_tab1 = vw_tab1.apply(highlight_rows, subset=[1], args=(names2,), axis=1)

ew_tex = ew_tab1.to_latex()
vw_tex = vw_tab1.to_latex()

# Replace '\background-coloraliceblue' with '\cellcolor{gray!25}'
ew_tex = ew_tex.replace("\\background-coloraliceblue", "\\cellcolor{lightgray}")
vw_tex = vw_tex.replace("\\background-coloraliceblue", "\\cellcolor{lightgray}")
ew_tex = ew_tex.replace("\\multirow[c]", " \\cline{1-13} \\multirow[c]")
vw_tex = vw_tex.replace("\\multirow[c]", " \\cline{1-13} \\multirow[c]")


#print(ew_tex)
print(vw_tex)

# Factor Models

In [ ]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import datetime as dt
from datetime import datetime
from pandas.tseries.offsets import *

# create a dataframe with best portfolios

def best_port(portfolios):
    Y = pd.DataFrame()

    # Itera attraverso ogni elemento nel dizionario
    for factor, ports in portfolios.items():
        if (factor == 'mj') or (factor == 'wmj'):
             port_key = 'port10'
        else:
            pos=factors.index(factor)
            # Scegli 'port1' o 'port10' in base al valore di 'factor'
            port_key = 'port10' if signs[pos] ==1  else 'port1'
    
        # Aggiungi i dati al DataFrame
        Y[factor] = ports[port_key]
    return Y

def multiple_regression_table(df, X_label, Y_label):
    Y=df[Y_label]
    X=df[X_label]
    # add constant on regressor matrix
    X = sm.add_constant(X)

    pvalues_df = pd.DataFrame()
    results_df = pd.DataFrame()
    R_list = []
    
    
    for y in Y.columns:
        # Perform the regression
        model = sm.OLS(Y[y], X)

        # Fit the model
        results = model.fit()
        coef = results.params
        t_stats = results.tvalues
        p = results.pvalues
        R = results.rsquared
        pvalues_df[y] = p 
        R_list.append(R)
        
    
        for x in X.columns:
            results_df.loc[x, y] = '{:.3f}'.format(coef[x])
            results_df.loc[f't stat {x}', y] = '({:.3f})'.format(t_stats[x])

    R_df = pd.DataFrame([R_list], columns=results_df.columns, index=['$R^2$'])
    
    def format_to_3f(x):
        return f"{x:.3f}"

    # Apply the function to the DataFrame
    R_df = R_df.applymap(format_to_3f)

    results_df = pd.concat([results_df, R_df], axis=0)
    # Create a LaTeX table from the results DataFrame
    latex_table = results_df.to_latex(escape=False)

    # Bold the t-statistics if the corresponding p-value is less than 0.05
    for y in Y.columns:
        for x in X.columns:
            if pvalues_df.loc[x, y] < 0.05:
                latex_table = latex_table.replace(f'{results_df.loc[f"{x}", y]}', f'\\textbf{{{results_df.loc[f"{x}", y]}}}')
                latex_table = latex_table.replace(f"t stat {x}", " ")
                
    return results_df, pvalues_df, latex_table, R_list

# equal - weighted
def create_df_factor_regression(portfolios):
    dataFF = pd.read_csv("./dataset_creation/original_data/FF_Research_Data_Factors_monthly.csv")
    mom = pd.read_csv("./dataset_creation/original_data/F-F_Momentum_Factor.csv")
    dataFF4F = pd.merge(dataFF, mom, on='date', how='inner')
    dataFF4F['date'] = pd.to_datetime(dataFF4F['date'],format='%Y%m')+pd.tseries.offsets.MonthEnd()
    dataFF4F['date'] = dataFF4F['date'] + MonthEnd(0)

    FFfactors=list(set(dataFF4F.columns)-set(['date']))
    dataFF4F[FFfactors]=dataFF4F[FFfactors]/100
    
    flat_portfolios=portfolios.reset_index()
    flat_portfolios.rename(columns={'medate':'date'},inplace=True)
    
    data_4fmodel=pd.merge(dataFF4F,flat_portfolios,on='date', how='inner')
    data_4fmodel.set_index('date',inplace=True)
    return data_4fmodel


In [ ]:
# MOMENTUM - FF portfolio
momFF = pd.read_csv("./dataset_creation/original_data/F-F_Momentum_Factor.csv")
momFF['date'] = pd.to_datetime(momFF['date'],format='%Y%m')+pd.tseries.offsets.MonthEnd()
momFF['date'] = momFF['date'] + MonthEnd(0)
momFF['MOM'] = momFF['MOM']/100
momFF=momFF.loc[(momFF.date >= date_initial) & (momFF.date <= date_final)].reset_index(drop = True)
#print(plt.plot(momFF['date'], momFF['MOM']))
print(momFF['MOM'].mean()*12)
momFF['cret_mom'] = (1 + momFF['MOM']).cumprod() - 1
plt.plot(momFF['date'], momFF['cret_mom'])
plt.show()

In [ ]:

df_regression=create_df_factor_regression(portfolios['mj'])

# CAPM
Y_label = portfolios['mj'].columns
X_label = ['Mkt_RF']
results_dfCAPM, pvalues_dfCAPM, latex_tableCAPM, R2 = multiple_regression_table(df_regression, X_label, Y_label)
latex_tableCAPM = latex_tableCAPM.replace("Mkt_RF", "Mkt-RF")
latex_tableCAPM = latex_tableCAPM.replace("const", "CAPM $\\alpha$")

print(latex_tableCAPM)

#3-Factor model
X_label = ['Mkt_RF', 'SMB', 'HML']
results_df3f, pvalues_df3, latex_table3, R3 = multiple_regression_table(df_regression, X_label, Y_label)
latex_table3 = latex_table3.replace("Mkt_RF", "Mkt-RF")
latex_table3 = latex_table3.replace("const", "3-factor $\\alpha$")

print(latex_table3)
#print(results_df3f)

# 4-Factor model
X_label = ['Mkt_RF', 'SMB', 'HML', 'MOM']
results_df, pvalues_df, latex_table, R4 = multiple_regression_table(df_regression, X_label, Y_label)
latex_table = latex_table.replace("Mkt_RF", "Mkt-RF")
latex_table = latex_table.replace("const", "4-factor $\\alpha$")

print(latex_table)
#results_df

In [ ]:
# value - weighted
# CAPM
df_regression=create_df_factor_regression(weighted_portfolios['wmj'])

X_label = ['Mkt_RF']
wresults_dfCAPM, wpvalues_dfCAPM, wlatex_tableCAPM, wR2 = multiple_regression_table(df_regression, X_label, Y_label)
wlatex_tableCAPM = wlatex_tableCAPM.replace("Mkt_RF", "Mkt-RF")
wlatex_tableCAPM = wlatex_tableCAPM.replace("const", "CAPM $\\alpha$")
print(wlatex_tableCAPM)

#3-Factor model
X_label = ['Mkt_RF', 'SMB', 'HML']
wresults_df3f, wpvalues_df3, wlatex_table3, wR3 = multiple_regression_table(df_regression, X_label, Y_label)
wlatex_table3 = wlatex_table3.replace("Mkt_RF", "Mkt-RF")
wlatex_table3 = wlatex_table3.replace("const", "3-factor $\\alpha$")

print(wlatex_table3)

# 4-Factor model
X_label = ['Mkt_RF', 'SMB', 'HML', 'MOM']
wresults_df, wpvalues_df, wlatex_table, wR4 = multiple_regression_table(df_regression, X_label, Y_label)
wlatex_table = wlatex_table.replace("Mkt_RF", "Mkt-RF")
wlatex_table = wlatex_table.replace("const", "4-factor $\\alpha$")

print(wlatex_table)